In [ ]:
!git clone https://github.com/maria-natale/ProgettoFVAB

fatal: destination path 'ProgettoFVAB' already exists and is not an empty directory.


In [ ]:
!python "/content/ProgettoFVAB/Code/create_datasetcsv_66.py"

100% 343/343 [00:08<00:00, 42.32it/s]
100% 274/274 [00:08<00:00, 32.79it/s]
(95900, 66)
100% 69/69 [00:00<00:00, 70.60it/s]
(24150, 66)
100% 274/274 [07:06<00:00,  1.56s/it]
100% 69/69 [01:05<00:00,  1.05it/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import numpy as np

path_git = '/content/ProgettoFVAB'
filename = '4_7'
path_drive = '/content/drive/MyDrive/Casillo&Natale'
dataset_dir = 'dataset_4_7/datasetCSV'
LANGUAGES = {4:'Spagnolo',
  7: 'Giapponese'}

MAX_LINES = 360

**Eseguire da qua**

In [ ]:

os.chdir(os.path.join(path_drive, dataset_dir, 'csv'))
x_train = pd.read_csv(filename+"_all_train.csv")
x_test = pd.read_csv(filename+"_all_test.csv")
y_train = pd.read_csv(filename+"targets_all_train.csv")
y_test = pd.read_csv(filename+"targets_all_test.csv")
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

number_train = 40/100 * x_train.shape[0]
number_test = 40/100 * x_test.shape[0]

(68250, 66)
(68250, 1)
(19250, 66)
(19250, 1)


**NO**

In [ ]:
print(x_train.shape)
print(x_test.shape)
x_train = x_train.loc[:number_train][:]
y_train = y_train.loc[:number_train][:]
x_test = x_test.loc[:number_test][:]
y_test = y_test.loc[:number_test][:]



(68250, 66)
(19250, 66)


In [ ]:
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

scaler = MinMaxScaler(feature_range=(0, 1))
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)


#Convert to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)
#Reshape the data into 3-D array
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

# Initialising the RNN
model = Sequential()
model.add(Bidirectional(LSTM(units = 100, return_sequences = True, input_shape = (x_train.shape[0], 1))))
model.add(Dropout(0.2))

# Adding a second LSTM layer and Dropout layer
model.add(Bidirectional(LSTM(units = 100, return_sequences = True)))
model.add(Dropout(0.2))

# Adding a third LSTM layer and Dropout layer
model.add(Bidirectional(LSTM(units = 100)))
model.add(Dropout(0.2))



# Adding the output layer
# For Full connection layer we use dense
model.add(Dense(8, activation='softmax'))


In [ ]:
import tensorflow as tf
#compile and fit the model on 30 epochs
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)



In [ ]:
model.fit(x_train, y_train, epochs = 2, batch_size = 50)
model.summary()

Epoch 1/2
 297/1365 [=====>........................] - ETA: 12:56 - loss: 0.7847 - accuracy: 0.5511

KeyboardInterrupt: ignored

# Nuova sezione

In [ ]:
#Convert x_test to a numpy array 
x_test = np.array(x_test)

#Reshape the data into 3-D array
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

In [ ]:
model.evaluate(x_test, y_test)

755/755 [==============================] - 63s 79ms/step - loss: 1.2599 - accuracy: 0.4612


[1.2599492073059082, 0.4611594080924988]

In [ ]:
from sklearn.metrics import confusion_matrix 

#check predicted values

predictions = model.predict_classes(x_test) 



/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
predictions

array([4, 4, 4, ..., 4, 4, 4])

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

multilabel_confusion_matrix(y_test, predictions, labels = [4, 7])


array([[[    0,  8050],
        [    0, 11200]],

       [[11200,     0],
        [ 8050,     0]]])